In [1]:
%pip install visualkeras

Note: you may need to restart the kernel to use updated packages.


In [2]:
import tensorflow as tf
from tensorflow import keras
from keras import backend as K
import numpy as np

2023-10-19 16:42:39.393508: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-10-19 16:42:43.301156: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [3]:
import os
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, BatchNormalization 
from keras.preprocessing import image
import cv2
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

In [4]:
data_path = "/home/ducna/Citrus/data_citrus_level/"
CATEGORIES = ["black_spot", "black_spot_early", "canker", "canker_early", "greening", "greening_early", "melanose", "melanose_early", "healthy"]
IMG_SIZE = 224

#create training data
training = []
def createTrainingData():
    for category in CATEGORIES:
        path = os.path.join(data_path, category)
        class_num = CATEGORIES.index(category)
        for img in os.listdir(path):
            img_array = cv2.imread(os.path.join(path,img))
            new_array = cv2.resize(img_array, (IMG_SIZE, IMG_SIZE))
            training.append([new_array, class_num])

createTrainingData()

In [5]:
import random
random.shuffle(training)

In [6]:
X = []
y = []
for features, label in training:
    X.append(features)
    y.append(label)

In [7]:
X = np.array(X).reshape(-1, IMG_SIZE, IMG_SIZE, 3)

#Normalize X
X = X.astype('float32')
X /= 255

Y = keras.utils.to_categorical(y,9)
print(X.shape)
print(Y.shape)   

(12480, 224, 224, 3)
(12480, 9)


In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size = 0.1)
test = X_test
X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size = 0.22)
print(X_train.shape)

(8760, 224, 224, 3)


In [9]:
#CNN1
cnn1 = Sequential()
cnn1.add(Conv2D(32, kernel_size = (3,3), padding = "same", activation="relu", input_shape = (224, 224, 3)))
cnn1.add(MaxPooling2D(pool_size=(2, 2)))
cnn1.add(BatchNormalization())
cnn1.add(Conv2D(32, kernel_size = (3,3), padding = "same", activation="relu"))
cnn1.add(MaxPooling2D(pool_size=(2, 2)))
cnn1.add(Dropout(0.5))
cnn1.add(Dense(256, activation = 'relu'))
cnn1.add(Dense(1, activation='softmax'))
cnn1.summary()

2023-10-18 08:48:37.892984: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1639] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 6914 MB memory:  -> device: 0, name: NVIDIA GeForce GTX 1080, pci bus id: 0000:03:00.0, compute capability: 6.1


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 224, 224, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2  (None, 112, 112, 32)      0         
 D)                                                              
                                                                 
 batch_normalization (Batch  (None, 112, 112, 32)      128       
 Normalization)                                                  
                                                                 
 conv2d_1 (Conv2D)           (None, 112, 112, 32)      9248      
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 56, 56, 32)        0         
 g2D)                                                            
                                                        

In [10]:
cnn1.compile(optimizer = 'Adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])

In [11]:
hist1 = cnn1.fit(X_train, y_train, batch_size = 16, epochs = 5, verbose = 1, validation_data = (X_test, y_test))

: 